# Reverts from SHA1 
Generates interactions.json

Format: interactions[user1][user2]:[[pageid,timestamp,deltatime,revision_id_reverted,revision_id_reverting], another revert, etc]

user1: user reverting
user2: user reverted
pageid: page_id 
timestamp: timestamp when the reverted version was created
deltatime: delta time from the reverted version to the reverting revision.
revision_id_reverted: revision id 

In [1]:
import mwreverts
import pandas as pd
from mwtypes import Timestamp



* revs_2017_users_more_than_10_revs.txt.gz was generated from this query: https://quarry.wmflabs.org/query/23424

In [2]:
df = pd.read_csv('revs_2017_users_more_than_10_revs.txt.gz',sep='\t')

In [3]:
interactions = {}
byPage = df.groupby('rev_page')


In [ ]:
print(len(byPage))
c = 0
for page, data in byPage :
    c+=1
    if c%100000==0:print(c)
    revs = [(info['rev_sha1'],(info['rev_user'],info['rev_timestamp'],info['rev_id'])) for info in data.sort_values('rev_timestamp').to_dict(orient='records')]
    for result in mwreverts.detect(revs):
        for user,timestamp,rev_id in result.reverteds:
            interactions[result.reverting[0]] = interactions.get(result.reverting[0],{})
            interactions[result.reverting[0]][int(user)] = interactions[result.reverting[0]].get(int(user),[])
            interactions[result.reverting[0]][int(user)].append([int(page),timestamp,Timestamp(str(result.reverting[1]))-Timestamp(str(timestamp)),result.reverting[2],rev_id])

In [5]:
import json
with open('interactions.json','w') as f:
    json.dump(interactions,f)